**IMPORT** LIBRARIES

In [1]:
import numpy as np 
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier     #KNN
from sklearn.metrics import accuracy_score

READ THE DATA:

- In this challenge you will use the dataset Titanic disaster.
- With the following lines of code, you can import the dataset form the file "train.csv", extract te column of labels (column "Survived") and read the infos of the other features.

you can also follow kaggle competition:
https://www.kaggle.com/code/amitkumarjaiswal/beginner-s-tutorial-to-titanic-using-scikit-learn/notebook

---



In [7]:
#from google.colab import drive
#if you downloaded the file in the Colab directory in your Google Drive, use the following lines of code
#drive.mount('/gdrive', force_remount=True)
Tr_file="T_db/train.csv" 
Ts_file="T_db/test.csv" 
dataset=pd.read_csv(Tr_file)
ts_ds=pd.read_csv(Ts_file)
X_tr=dataset.drop('Survived',axis=1)
X_ts= dataset.drop('Survived',axis=1)

Y_tr=dataset.Survived
Y_ts=dataset.Survived

X_tr.info()
Y_tr



FileNotFoundError: [Errno 2] No such file or directory: '/gdrive/My Drive/Colab Notebooks/T_db/test.csv'

FEATURES ENGINEERING:

- Evaluate if there are features with missing values and modify them (see https://scikit-learn.org/stable/modules/impute.html)
- Find meaningful features (some features are not needed because do not contain useful infos)
- Encode categorical features with numerical values (see https://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features)

**MISSING VALUE WITH A FIXED VALUE** 

**MISSING VALUE WITH MEAN VALUE AND MOST FREQUENT VALUE** 

**CATEGORICAL FEATURES**

*MEANINGFUL FEATURES - SELECTION - EXTRACTION -

In [ ]:
#DO NOT MODIFY THE CLASSIFIER AND THE PERFORMANCE EVALUATION

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_tr,Y_tr)
scores = knn.predict(X_ts)

# Show prediction accuracy
print('\nPrediction accuracy:')
print('{:.2%}\n'.format(accuracy_score(Y_ts, scores)))

